#14757 Homework 1  (150 pts)

## **Due:** Wednesday September 18 at 3pm ET / 12 noon PT

## Submission Instructions

*   Download your completed notebook by clicking File->Download .ipynb and submit it on Gradescope
*   Check your submission on Gradescope to make sure that all your code, code output and written responses appear correctly

## Problem 1: Gradescope Autograder Placeholder (0 pts)

Gradescope requires that problem 1 be autograded for code submissions, but there are no autograded problems. Continue to problem 2.

## Problem 2: The Wrongful Conviction of Sally Clark (20 pts)

Sally Clark was a British lawyer who was wrongly sentenced to life in prison in 1999 for the deaths of her two infant children. Her elder son Christopher died at age 11 weeks in December 1996 and her younger son Harry at 8 weeks in January 1998. At her trial, the defence argued that the deaths were due to sudden infant death syndrome (SIDS). Clark was convicted on the basis of testimony by pediatrician Sir Roy Meadow, who made the following argument:

*   Hospital records show that the ratio of SIDS deaths to live births in affluent non-smoking families is about $\frac{1}{8500}$. (A live birth is a birth in which a child is born alive; not a still birth.)
*   The chance of two SIDS deaths occurring in the same family is about $\frac{1}{8500^2} \approx \frac{1}{73000000}$.
*   It is therefore extremely unlikely that Clark is innocent.

As a result of this prosecution, Clark spent more than 3 years in prison and was finally exonerated in 2003 after it was determined that Meadow’s expert testimony was flawed. Two other women against whom Meadow provided expert testimony had their convictions overturned as well.



**2.1** (10 pts) Identify a flaw in Meadow’s $\frac{1}{73000000}$ figure.

WRITE YOUR ANSWER HERE:
The problem with Meadow's figure is that he assumes conditional independence meaning that the 2 infant deaths are uncorrelated such that the probability of 2x SIDS deaths in the same family is 1/8500. However, Meadows being a pediatrician should recognize that 2 children being afflicted with the same disease coming from the same is not conditionally independent, and therefore his assumption would fail thus rendering his figure completely incorrect.

**2.2** (10 pts) Even if we accept Meadow’s $\frac{1}{73000000}$ calculation as valid, what is wrong with a juror interpreting it as the probability of Clark’s innocence?

WRITE YOUR ANSWER HERE:
Even if we accept Meadow's Number as valid, the juror should not be concerned with Meadow's Number as his calculation gives the probability of 2x SIDS deaths given Clark's innocence P(2x SIDS Deaths | Clark's Innocence). First and foremost, the juror should be concerned with the probability that Clark is innocent given the two natural infant deaths regardless of cause, and that equation would be {P(2x Infant Deaths | Innocence) * P(Innocence)/P(2x Natural Infant Deaths)}. It stands to reason that $P(2x SIDS Deaths | Clark's Innocence) != P(Innocence | 2x Natural Infant Deaths)$


## Problem 3: Attacking a Multinomial Naive Bayes Spam Filter (80 pts)



In this problem you will execute several attacks on a naive Bayes spam filter for text messages (SMS). Run the cell below to set up the spam filter. The code performs the following steps:

*   Reading and displaying the data
*   Splitting the data into training and testing sets
*   Converting each text message into a vector of word counts, so that the training messages are represented as a single matrix, and so are the test messages
*   Training the spam filter using the training set and evaluating it with respect to the testing set

Helpful documentation:

*   https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html
*   https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html

This implementation is adapted from source code here: https://github.com/rahulvasaikar/SMS-spam-Detection

The original data is documented here: https://archive.ics.uci.edu/ml/datasets/SMS+Spam+Collection

In [ ]:
# READING AND DISPLAYING DATA

# Reading tab separated text file into a Pandas dataframe
import pandas as pd
url = 'http://www.andrew.cmu.edu/user/dvaroday/14757/data/hw1/SMSSpamCollection'
df = pd.read_table(url, sep='\t', names=['label', 'message'])

# Mapping ham/spam labels to 0/1
df['binary_label'] = df.label.map({'ham':0,'spam':1})

# Display dataframe
print('DATASET')
display(df)

# SPLITTING INTO TRAINING AND TESTING SETS

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df['message'],
                                        df['binary_label'], random_state=1)

total_rows = df.shape[0]
training_rows = X_train.shape[0]
test_rows = X_test.shape[0]

print('\nDATA SPLIT STATISTICS')
print('Number of rows in the total set:    {}'.format(total_rows))
print('Number of rows in the training set: {}'.format(training_rows))
print('Number of rows in the test set:     {}'.format(test_rows))

# CONVERTING EACH TEXT MESSAGE INTO A VECTOR OF WORD COUNTS

from sklearn.feature_extraction.text import CountVectorizer
# Instantiate the CountVectorizer
count_vector = CountVectorizer()

# Fit the training data
# Returns matrix of feature counts in training text messages
# Features are obtained from the training set itself
training_data = count_vector.fit_transform(X_train)

# Transform testing data
# Returns matrix of feature counts in test text messages
# Note that the features are the ones obtained from the training set
testing_data = count_vector.transform(X_test)

# TRAINING AND EVALUATING MODEL

# Train
from sklearn.naive_bayes import MultinomialNB
# MultinomialNB with Laplacian smoothing (alpha=1.0)
spam_filter = MultinomialNB(alpha=1.0)
spam_filter.fit(training_data, y_train)

# Evaluate
predictions = spam_filter.predict(testing_data)
from sklearn.metrics import accuracy_score, recall_score
accuracy = accuracy_score(y_test, predictions)
recall = recall_score(y_test, predictions)

print('\nPERFORMANCE')
print('Accuracy:', "{:.3f}".format(accuracy))
print('Recall:  ', "{:.3f}".format(recall))

### Understanding baseline performance

**3.1** (10 pts) In the cell above *accuracy* is the fraction of messages that are correctly classified by the spam filter. *Recall* is the fraction of true spam messages correctly classified as spam. But the message recipient cares about true ham messages that are classified as spam because these messages are filtered out by the spam filter. Neither accuracy nor recall address this issue directly. Instead, the metric *specificity* represents the fraction of true ham messages that the recipient will actually receive. Fix the code between the comments in the cell below to calculate specificity for the test set.

https://en.wikipedia.org/wiki/Precision_and_recall

https://en.wikipedia.org/wiki/Sensitivity_and_specificity

In [ ]:
def specificity_score(y_test, predictions):
  test_messages = len(y_test.values)
  true_spams = sum(y_test.values)
  positives = sum(predictions)
  true_positives = sum(y_test.values * predictions)

  # START EDITING HERE - DON'T REMOVE THIS COMMENT
  return 0
  # END EDITING HERE - DON'T REMOVE THIS COMMENT

specificity = specificity_score(y_test, predictions)
print('Specificity:', "{:.3f}".format(specificity))

### Evasion attack

**3.2** (10 pts) In this part you act as a spammer who wants to evades the spam filter. Edit `evasion_message` in the cell below so that its probability of being labeled spam becomes less than 1%. For full credit, `evasion_message` must convey the same meaning as `original_message`.

In [ ]:
original_message = 'Txt back to get your $1000 prize now'

# EDIT THE MESSAGE BELOW - DON'T REMOVE THIS COMMENT
evasion_message = 'Text me back now to receive the contest prize money that you are owed in the amount of one thousand dollars'

# Probabilistic predictions of original_message and evasion_message
df_evasion = pd.DataFrame([original_message, evasion_message],
                          columns=['message'])
message_data = count_vector.transform(df_evasion['message'])
prediction_probabilities = spam_filter.predict_proba(message_data)

print('Probability that original message is spam: ',
      "{:.6f}".format(prediction_probabilities[0,1]))
print('Probability that evasion message is spam:  ',
      "{:.6f}".format(prediction_probabilities[1,1]))

**3.3** (10 pts) Describe the process you used to create your `evasion_message` in 1-2 sentences. Was this an open box (whitebox) or closed box (blackbox) attack? If open box, specify what information about `spam_filter` you used.

WRITE YOUR ANSWER HERE:

### Poisoning attack

**3.4** (10 pts) In this part you play the role of an adversary working against a ride share company. The company wants to deliver `target_message` to its customers. You gain access to the training set and have the opportunity to poison the training data with one poison message/label pair, but your example may be rejected by the spam filter administrator. Edit `poison_message` and/or `poison_label` so that they are accepted by the administrator and make the probability of `target_message` being labeled spam greater than 50%.

In [ ]:
target_message = 'Your ride share has arrived'
df_target = pd.DataFrame([target_message], columns=['message'])
target_data = count_vector.transform(df_target['message'])
target_probabilities = spam_filter.predict_proba(target_data)
print('Probability that target message is spam before poisoning:',
      "{:.6f}".format(target_probabilities[0,1]))

# START EDITING HERE - DON'T REMOVE THIS COMMENT
poison_message = ''
poison_label = 0
# END EDITING HERE - DON'T REMOVE THIS COMMENT

# Administrator runs poison message through original spam filter
# to validate its label
df_poison = pd.DataFrame([[poison_message, poison_label]],
                         columns=['message', 'binary_label'])
poison_data = count_vector.transform(df_poison['message'])
poison_validation = spam_filter.predict(poison_data)

if poison_label != poison_validation[0]:

  print('Administrator rejects your poison message/label')

else:

  print('Administrator accepts your poison message/label')

  # Initialize retraining data with original training data and append poison
  df_retrain = pd.DataFrame(X_train, columns=['message'])
  df_retrain['binary_label'] = y_train.values
  df_retrain = pd.concat([df_retrain, df_poison], ignore_index=True)

  # Retrain
  recount_vector = CountVectorizer()
  retraining_data = recount_vector.fit_transform(df_retrain['message'])
  retrained_filter = MultinomialNB(alpha=1.0)
  retrained_filter.fit(retraining_data, df_retrain['binary_label'])

  # Evaluate target message with retrained_filter
  recount_target_data = recount_vector.transform(df_target['message'])
  poisoned_probabilities = retrained_filter.predict_proba(recount_target_data)

  print('Probability that target message is spam after poisoning: ',
        "{:.6f}".format(poisoned_probabilities[0,1]))

**3.5** (10 pts) Describe an additional way for the administrator to decide whether to reject the poison message/label pair in order to defend against this type of attack. Use 1-4 sentences in your answer.

WRITE YOUR ANSWER HERE:

### Reverse engineering attack

**3.6** (20 pts) In this part you will perform a reverse engineering attack against the spam filter administrator. Your goal is to reconstruct the `spam_filter` model without using any of the original training messages or labels. In the starter code below the attacker sends a set of `probe_messages` to the `spam_filter`. The attacker can detect whether each message has been classified as spam or ham by inspecting its read receipt. The attacker uses these classifications as labels to train `reconstructed_filter`.

Edit `probe_messages` so that `reconstructed_filter` achieves at least 97% accuracy and 88% recall on the test set. You can use at most `max_probes = 5000` probe messages. You may **not** use the training or test sets to help construct `probe_messages`, **except** for the list of words provided to you in the list `vocab`.

In [ ]:
max_probes = 5000
vocab = count_vector.get_feature_names_out()

# START EDITING HERE - DON'T REMOVE THIS COMMENT
probe_messages = ['probe ' + str(i) for i in range(max_probes)]
# END EDITING HERE - DON'T REMOVE THIS COMMENT

print('Number of probe messages:', len(probe_messages))

# Run probe_messages through spam_filter to obtain probe_labels
df_probes = pd.DataFrame(probe_messages, columns=['message'])
probe_data = count_vector.transform(df_probes['message'])
probe_labels = spam_filter.predict(probe_data)

# Train reconstructed_filter using probe_messages and probe_labels
reconstructed_filter = MultinomialNB(alpha=1.0)
reconstructed_filter.fit(probe_data, probe_labels)
reconstructed_predictions = reconstructed_filter.predict(testing_data)

# Evaluate
accuracy = accuracy_score(y_test, reconstructed_predictions)
recall = recall_score(y_test, reconstructed_predictions)

print('Accuracy:', "{:.3f}".format(accuracy))
print('Recall:  ', "{:.3f}".format(recall))

**3.7** (10 pts) Describe one way that the administrator can defend against this type of attack. Use 1-4 sentences in your answer.

WRITE YOUR ANSWER HERE:

## Problem 4: Implementing a Multinomial Naive Bayes Classifier (50 pts)

In this problem you will implement a Multinomial Naive Bayes classifier from scratch that performs identically to the one from the `sklearn.naive_bayes` library you used above. First run the cell below to load the preprocess the data.

In [ ]:
# Reading tab separated text file into a Pandas dataframe
import pandas as pd
url = 'http://www.andrew.cmu.edu/user/dvaroday/14757/data/hw1/SMSSpamCollection'
df = pd.read_table(url, sep='\t', names=['label', 'message'])

# Mapping ham/spam labels to 0/1
df['binary_label'] = df.label.map({'ham':0,'spam':1})

# Splitting into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df['message'],
                                        df['binary_label'], random_state=1)

# Converting each text message into a vector of word counts
from sklearn.feature_extraction.text import CountVectorizer
count_vector = CountVectorizer()
training_data = count_vector.fit_transform(X_train)
testing_data = count_vector.transform(X_test)

In the cell below complete the class `MultinomialNB` by implementing the `fit()` and `predict()` methods. The constructor `__init__()` is written for you but you can modify it according to your needs.

For full credit your implementation should not use any loops. Instead you should operate directly on vectors and matrices, not iterate through them. In particular your `fit()` method should calculate log-probability ratios with `np.log()` so that your `predict()` method can use `np.matmul()` for matrix multiplication. For your convenience the starter code already converts the method inputs into `numpy` arrays. Here's a guide to vectorization and broadcasting with `numpy`: https://realpython.com/numpy-array-programming

Make the following assumptions:

*   The classification task is binary. Specifically `train_labels` is a vector of values 0 and 1 only.
*   The feature values in `train_features` and `test_features` are nonnegative integers.

In [ ]:
import numpy as np

# START EDITING HERE - DON'T REMOVE THIS COMMENT

class MultinomialNB:

  def __init__(self, alpha):
    self.alpha = alpha # Equal to 1.0 for Laplace smoothing
    self.label_log_prob_ratio = None
    self.feature_log_prob_ratios = None

  def fit(self, train_features, train_labels):
    # Converting inputs to numpy arrays
    train_features = np.array(train_features.todense())
    train_labels = np.array(train_labels.values)

  def predict(self, test_features):
    # Converting input to numpy arrays
    test_features = np.array(test_features.todense())
    # Initializing return value
    preds = np.zeros(test_features.shape[0], dtype=np.uint8)
    return preds

# END EDITING HERE - DON'T REMOVE THIS COMMENT



### DO NOT EDIT THE CODE BELOW THIS LINE

# Train
# MultinomialNB with Laplacian smoothing (alpha=1.0)
spam_filter = MultinomialNB(alpha=1.0)
spam_filter.fit(training_data, y_train)

# Evaluate
predictions = spam_filter.predict(testing_data)
from sklearn.metrics import accuracy_score, recall_score
accuracy = accuracy_score(y_test, predictions)
recall = recall_score(y_test, predictions)
print('\nPERFORMANCE')
print('Accuracy:', "{:.3f}".format(accuracy))
print('Recall:  ', "{:.3f}".format(recall))